In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.sparse
pd.set_option("display.max_columns",101)
RANDOM_STATE = 42

In [2]:
DATA_PATH = "../data/instacart/"

In [3]:
def save_sparse_matrix(filename, x):
    x_coo = x.tocoo()
    row = x_coo.row
    col = x_coo.col
    data = x_coo.data
    shape = x_coo.shape
    np.savez(filename, row=row, col=col, data=data, shape=shape)

def load_sparse_matrix(filename):
    y = np.load(filename)
    z = scipy.sparse.coo_matrix((y['data'], (y['row'], y['col'])), shape=y['shape'])
    return z

save_file = DATA_PATH + "user_item.npz"

In [4]:
load_S = load_sparse_matrix(save_file)
user_item_matrix = scipy.sparse.csr_matrix(load_S)

#### inverse_matrix

#### implement simple LFM model

In [5]:
item_pool = range(0, load_S.shape[1])

def random_select_negative_sample(items):
    import random
    ret = {}
    
    result = sum(items.values())
    for item, value in items.iteritems():
        ret[item] = value / float(result)
    
    n = 0
    for i in xrange(3 * len(items)):
        item = item_pool[random.randint(0, len(item_pool) - 1)]
        
        if item in items:
            continue
        
        ret[item] = 0
        
        n += 1
        if n >= len(items):
            break
    return ret

In [6]:
ret = random_select_negative_sample({1: 2, 2: 3})

In [7]:
ret

{1: 0.4, 2: 0.6, 22158: 0, 33670: 0}

In [8]:
xx = user_item_matrix.todok()

In [9]:
print xx.shape

(206209, 49677)


In [10]:
for i in xx.iteritems():
    print i 
    break

((97555, 46216), 13)


In [11]:
len(list(xx.iteritems()))

9098126

In [12]:
dok_matrix = user_item_matrix.todok()

In [13]:
def make_user_item_dict(X):
    from collections import defaultdict
    result = defaultdict(dict)
    for user_item, value in dok_matrix.iteritems():
        user_id, item_id = user_item
        result[user_id][item_id] = value
    return result

In [14]:
user_items = make_user_item_dict(dok_matrix)

In [15]:
u_items = user_items.items()

#### tuning the parameter, because parameter is too small, C should be larger

In [77]:
def LatentFactorModel(ui_data, n_factor, n_step, alpha, C):
    num_user, num_item = user_item_matrix.shape
    P_uk = np.ones((num_user, n_factor)) / float(n_factor)
    Q_ik = np.ones((num_item, n_factor)) / float(n_factor)
    for step in xrange(n_step):
        num_error = 0
        i = 0
        for user_id, items in ui_data.iteritems():
            i += 1
            if i % 10000 == 0:
                print 'count user:', i
            samples = random_select_negative_sample(items)
            for item_id, value in samples.iteritems():
                E_ui = value - P_uk[user_id, :].dot(Q_ik[item_id, :].transpose())
                old_error = E_ui * E_ui
                P_uk[user_id, :] = P_uk[user_id, :] + alpha * (Q_ik[item_id, :] * E_ui - C * P_uk[user_id, :])
                Q_ik[item_id, :] = Q_ik[item_id, :] + alpha * (P_uk[user_id, :] * E_ui - C * Q_ik[item_id, :])
                new_error = (value - P_uk[user_id, :].dot(Q_ik[item_id, :].transpose())) ** 2
                
                num_error += new_error
        alpha *= 0.9
    
        print "step:", step, "error:", num_error
    return P_uk, Q_ik

P, Q = LatentFactorModel(user_items, 5, 1, 0.5, 0.1)

count user: 10000
count user: 20000
count user: 30000
count user: 40000
count user: 50000
count user: 60000
count user: 70000
count user: 80000
count user: 90000
count user: 100000
count user: 110000
count user: 120000
count user: 130000
count user: 140000
count user: 150000
count user: 160000
count user: 170000
count user: 180000
count user: 190000
count user: 200000
step: 0 error: 13521.9864463


In [17]:
df = pd.DataFrame(P)

In [78]:
q_df = pd.DataFrame(Q)
q_df.describe()

,0,1,2,3,4
count,4.967700e+04,4.967700e+04,4.967700e+04,4.967700e+04,4.967700e+04
mean,2.399242e-03,2.399242e-03,2.399242e-03,2.399242e-03,2.399242e-03
std,5.425784e-03,5.425784e-03,5.425784e-03,5.425784e-03,5.425784e-03
min,2.475507e-12,2.475507e-12,2.475507e-12,2.475507e-12,2.475507e-12
25%,5.760345e-05,5.760345e-05,5.760345e-05,5.760345e-05,5.760345e-05
50%,5.535156e-04,5.535156e-04,5.535156e-04,5.535156e-04,5.535156e-04
75%,2.585277e-03,2.585277e-03,2.585277e-03,2.585277e-03,2.585277e-03
max,2.205712e-01,2.205712e-01,2.205712e-01,2.205712e-01,2.205712e-01


In [79]:
Q

array([[  1.05470361e-02,   1.05470361e-02,   1.05470361e-02,
          1.05470361e-02,   1.05470361e-02],
       [  4.32630341e-04,   4.32630341e-04,   4.32630341e-04,
          4.32630341e-04,   4.32630341e-04],
       [  6.82226344e-03,   6.82226344e-03,   6.82226344e-03,
          6.82226344e-03,   6.82226344e-03],
       ..., 
       [  1.59759219e-03,   1.59759219e-03,   1.59759219e-03,
          1.59759219e-03,   1.59759219e-03],
       [  1.56898165e-05,   1.56898165e-05,   1.56898165e-05,
          1.56898165e-05,   1.56898165e-05],
       [  1.37736710e-03,   1.37736710e-03,   1.37736710e-03,
          1.37736710e-03,   1.37736710e-03]])

#### Accoring to P and Q, we try to calculate result

In [19]:
orders = pd.read_csv(DATA_PATH + "orders.csv")

#### Get train use-item_list

In [20]:
train = pd.read_csv(DATA_PATH + "order_products__train.csv")

In [21]:
train_order_user = pd.merge(train, orders, on='order_id')

In [22]:
train_user_products = pd.DataFrame(train_order_user.groupby('user_id')['product_id'].apply(list))

In [23]:
train_user_products.head()

,product_id
user_id,
1,"[196, 25133, 38928, 26405, 39657, 10258, 13032..."
2,"[22963, 7963, 16589, 32792, 41787, 22825, 1364..."
5,"[15349, 19057, 16185, 21413, 20843, 20114, 482..."
7,"[12053, 47272, 37999, 13198, 43967, 40852, 176..."
8,"[15937, 5539, 10960, 23165, 22247, 4853, 27104..."


In [24]:
prior = pd.read_csv(DATA_PATH + "order_products__prior.csv")
order_user = pd.merge(prior, orders, on='order_id')
prior_user_products = pd.DataFrame(order_user.groupby('user_id')['product_id'].apply(list))
def cal_dict(x):
    from collections import Counter
    a = Counter(x)
    return dict(a)

prior_user_products_dict = pd.DataFrame(prior_user_products['product_id'].apply(cal_dict))
product_id_list = np.unique(order_user.product_id)
product_id_dict = {}
for idx, product_id in enumerate(product_id_list):
    product_id_dict[product_id] = idx
    
user_id_list = np.unique(order_user.user_id)
user_id_dict = {}
for idx, user_id in enumerate(user_id_list):
    user_id_dict[user_id] = idx

In [25]:
def cal_dict(x):
    from collections import Counter
    a = Counter(x)
    return dict(a)

train_user_products_dict = pd.DataFrame(train_user_products['product_id'].apply(cal_dict))

#### Accoring to train_data, calculate the threshold for all provided data

In [26]:
def predict(train_dict, P_uk, Q_ik, user_dict, item_dict):
    from collections import defaultdict
    result = defaultdict(dict)
    for x in train_dict['product_id'].iteritems():
        user_id, product_dict = x[0], x[1]
        for product_id, num in product_dict.iteritems():
            if product_id not in item_dict:
                continue
            user_idx = user_dict[user_id]
            item_idx = item_dict[product_id]
            result[user_id][product_id] = P_uk[user_idx, :].dot(Q_ik[item_idx, :].transpose())
    return result

In [80]:
train_result = predict(prior_user_products_dict, P, Q, user_id_dict, product_id_dict)

In [28]:
def normalize(data):
    result = {}
    for user_id, product_dict in data.iteritems():
        cnt = sum(product_dict.values())
        result[user_id] = {}
        for product_id, value in product_dict.iteritems():
            result[user_id][product_id] = value / cnt
    return result

normal_data = normalize(train_result)

In [81]:
def score_f(predict_list, product_list):
    recall = 0
    precision = 0
    
    same_one = 0
    for product_id in predict_list:
        if product_id in product_list:
            same_one += 1
    
    if len(predict_list) == 0:
        return 0
    
    recall = float(same_one) / len(product_list)
    precision = float(same_one) / len(predict_list)
    
    if recall == 0 and precision == 0:
        return 0
    
    return 2 * (recall * precision) / (recall + precision)

In [41]:
user_order_list = orders.groupby('user_id')['order_id'].apply(list)
user_order_dict = user_order_list.to_dict()
cart_number_dict = order_user.groupby('order_id')['add_to_cart_order'].apply(list)
unique_cart_number_list = cart_number_dict.apply(np.unique)
unique_cart_number_dict = unique_cart_number_list.to_dict()

In [46]:
average_length_dict = {}
i = 0
for user_id, order_list in user_order_dict.iteritems():
    i += 1
    order_length_list = [len(unique_cart_number_dict[order_id]) for order_id in order_list if order_id in unique_cart_number_dict]
    average_length_dict[user_id] = max(1, sum(order_length_list) / len(order_length_list))

In [47]:
len(average_length_dict)

206209

#### Use theshold

In [82]:
def cal_result(norm_data, train_dict, theshold):
    f_score = 0
    for x in train_dict['product_id'].iteritems():
        user_id, product_dict = x[0], x[1]
        
        product_list = list(product_dict.keys())
        predict_dict = normal_data[user_id]
        predict_list = []
        for key, value in predict_dict.iteritems():
            if value >= theshold:
                predict_list.append(key)
                
        f_score += score_f(predict_list, product_list)
    
    f_score /= float(len(train_dict))
    return f_score

In [83]:
cal_result(normal_data, train_user_products_dict, 0.0032)

0.20160279283468824

In [84]:
cal_result(normal_data, train_user_products_dict, 0.0034)

0.2017250967104473

In [85]:
cal_result(normal_data, train_user_products_dict, 0.0035)

0.20179577395140882

In [94]:
cal_result(normal_data, train_user_products_dict, 0.005)

0.2022545690099914

In [53]:
def cal_result(norm_data, train_dict, len_dict):
    f_score = 0
    for x in train_dict['product_id'].iteritems():
        user_id, product_dict = x[0], x[1]
        
        product_list = list(product_dict.keys())
        value_dict = normal_data[user_id]
        
        len_user_id = len_dict[user_id]
        predict_list = sorted(value_dict.keys(), key=lambda x: value_dict[x], reverse=True)[:len_user_id]
        
        f_score += score_f(predict_list, product_list)
    
    f_score /= float(len(train_dict))
    return f_score

In [54]:
cal_result(normal_data, train_user_products_dict, average_length_dict)

0.16193138948670194

#### Use 

In [55]:
test_user_id_list = pd.DataFrame(orders.groupby('eval_set')['user_id'].apply(list))

In [56]:
unique_user_id_list = np.unique(test_user_id_list.user_id['test'])

In [460]:
len(unique_user_id_list)

75000

In [95]:
def predict_result(norm_data, user_id_list, theshold=0.005):
    result = {}
    for user_id in user_id_list:
        predict_dict = normal_data[user_id]
        predict_list = []
        for key, value in predict_dict.iteritems():
            if value >= theshold:
                predict_list.append(key)
                
        result[user_id] = predict_list
    
    return result

In [98]:
def predict_result_2(norm_data, user_id_list, len_dict):
    result = {}
    for user_id in user_id_list:
        value_dict = normal_data[user_id]
        
        len_user_id = len_dict[user_id]
        predict_list = sorted(value_dict.keys(), key=lambda x: value_dict[x], reverse=True)[:len_user_id]
                
        result[user_id] = predict_list
    
    return result

In [100]:
test_result = predict_result_2(normal_data, unique_user_id_list, average_length_dict)

In [101]:
test_order_data = orders.ix[orders['eval_set'] == 'test']

In [102]:
test_order_data.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [465]:
test_order_data.iteritems()

<generator object iteritems at 0x4771aeaa0>

In [103]:
print_data = []
for x in test_order_data.iterrows():
    order_id = x[1].order_id
    user_id = x[1].user_id
    print_data.append((order_id, test_result[user_id]))

In [104]:
sorted_print_data = sorted(print_data)

In [105]:
sorted_print_data[0]

(17, [47766, 21903, 31964, 16965, 40002, 44056])

In [ ]:
model_id_list = [x[0] for x in sorted_print_data]
product_id_data = [" ".join(map(str, x[1])) for x in sorted_print_data]

In [65]:
data = pd.DataFrame({'order_id': model_id_list, "products": product_id_data})

In [66]:
data.head()

,order_id,products
0,17,26429 40002 16965 11494 13107 39275 21709 2190...
1,34,26369 36994 20995 7559 18441 651 39180 5134 31...
2,137,42625 9515 28934 22281 7179 5134 27663 7952 47...
3,182,34824 26128 27156 17948 2078 48682 14891 23645...
4,257,1025 24964 47766 24838 6795 29837 37646 39055 ...


In [67]:
data.to_csv(DATA_PATH + "predict.csv", index=False)